user_ID가 6,7로 끝나는 경우 검증 데이터 / 8,9로 끝나는 경우 테스트 데이터

In [ ]:
import os
import pandas as pd

# DataFrame 초기화
df = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz', 'ax', 'ay', 'az', 'label'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D02': 'walking',
    'D03': 'jogging',
    'D04': 'jogging',
    'D05': 'stairs_walking',
    'D06': 'stairs_walking',
    'D07': 'standing-sitting',
    'D08': 'standing-sitting',
    'F01': 'fall',
    'F02': 'fall',
    'F06': 'fall',
    'F05': 'fall'
}

# 데이터 폴더 경로
data_folder = '../data/SisFall_dataset'

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        # user_ID가 'SA' 또는 'SE'로 시작하고 뒤에 오는 숫자가 7이거나 8인 경우 건너뛰기
        if user_ID.startswith('SA') or user_ID.startswith('SE'):
            numeric_part = user_ID[2:]
            if user_ID.startswith('SA') or user_ID.startswith('SE'):
                numeric_part = user_ID[2:]
                if numeric_part.isdigit() and int(numeric_part) >= 6 and int(numeric_part) <= 9:
                    continue


        for file_name in os.listdir(dir_path):
            if file_name.endswith('.txt'):
                # 파일 경로
                file_path = os.path.join(dir_path, file_name)

                # 파일 이름에 대한 정보
                activity, _, trial_ID_with_extension = file_name.split('_')
                trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                if activity in ['D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'F01', 'F02', 'F05', 'F06']:
                    # task_id에 해당하는 label 정보
                    task_ID = activity
                    label = label_mapping[task_ID]

                    #Angular velocity 변환을 위한 공식에 사용되는 값들
                    Range = 2000  # 예시 값을 사용
                    Resolution = 16  # 예시 값을 사용

                    range_g = 16  # range for ADXL345 when set to ±16g
                    resolution_bits = 13  # resolution for ADXL345

                    # 파일에서 데이터를 읽어와서 처리
                    with open(file_path, 'r', encoding='utf-8') as file:
                        lines = file.readlines()

                    # 데이터 파싱 및 계산
                    gx_values = []
                    gy_values = []
                    gz_values = []

                    ax_values = []
                    ay_values = []
                    az_values = []

                    # 데이터 파싱 및 DataFrame에 추가
                    for i, line in enumerate(lines):
                        if i % 5 == 0:  # 10의 배수인 행만 처리
                            # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                            fields = line.strip().split(',')

                            if len(fields) >= 6:  # 필드 수 확인
                                # gx, gy, gz 값 얻기 및 변환 계산
                                gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                                gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                                gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                                ax_value = (2 * range_g / (2 ** resolution_bits)) * float(fields[0])
                                ay_value = (2 * range_g / (2 ** resolution_bits)) * float(fields[1])
                                az_value = (2 * range_g / (2 ** resolution_bits)) * float(fields[2])


                                gx_values.append(gx_value)
                                gy_values.append(gy_value)
                                gz_values.append(gz_value)

                                ax_values.append(ax_value)
                                ay_values.append(ay_value)
                                az_values.append(az_value)

                                if len(gx_values) == 1:
                                    # 한 줄의 레코드 처리가 완료되었으므로 DataFrame에 추가
                                    df.loc[len(df)] = {
                                        'user_ID': user_ID,
                                        'trial_ID': trial_ID,
                                        'task_ID': task_ID,
                                        'gx': gx_values[0],
                                        'gy': gy_values[0],
                                        'gz': gz_values[0],
                                        'ax': ax_values[0],
                                        'ay': ay_values[0],
                                        'az': az_values[0],
                                        'label': label
                                    }
                                    record_count += 1

                                    gx_values = []
                                    gy_values = []
                                    gz_values = []

                                    ax_values = []
                                    ay_values = []
                                    az_values = []


print(df)

from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 인스턴스 생성
scaler = MinMaxScaler(feature_range=(-1, 1))

# 'gx', 'gy', 'gz' 열의 데이터 추출
data = df[['gx', 'gy', 'gz','ax', 'ay', 'az']].values

# 데이터 스케일링
scaled_data = scaler.fit_transform(data)

# 스케일링된 값을 새로운 열에 저장
df['scaled_gx'] = scaled_data[:, 0]
df['scaled_gy'] = scaled_data[:, 1]
df['scaled_gz'] = scaled_data[:, 2]
df['scaled_ax'] = scaled_data[:, 3]
df['scaled_ay'] = scaled_data[:, 4]
df['scaled_az'] = scaled_data[:, 5]

print(df.head())

# CSV 파일로 저장
csv_path = '../data/SisFall_train_L_5.csv'
df.to_csv(csv_path, index=False)
print('CSV 파일이 저장되었습니다:', csv_path)

In [1]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 인스턴스 생성
scaler = MinMaxScaler(feature_range=(-1, 1))

# 'gx', 'gy', 'gz' 열의 데이터 추출
data = df[['gx', 'gy', 'gz']].values

# 데이터 스케일링
scaled_data = scaler.fit_transform(data)

# 스케일링된 값을 새로운 열에 저장
df['scaled_gx'] = scaled_data[:, 0]
df['scaled_gy'] = scaled_data[:, 1]
df['scaled_gz'] = scaled_data[:, 2]

print(df.head())

NameError: name 'df' is not defined

In [15]:
print(df['user_ID'].unique())

In [10]:
# CSV 파일로 저장
csv_path = '../data/SisFall_train_L.csv'
df.to_csv(csv_path, index=False)
print('CSV 파일이 저장되었습니다:', csv_path)

In [2]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# DataFrame 초기화
df_val = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz', 'ax', 'ay', 'az', 'label'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D02': 'walking',
    'D03': 'jogging',
    'D04': 'jogging',
    'D05': 'stairs_walking',
    'D06': 'stairs_walking',
    'D07': 'standing-sitting',
    'D08': 'standing-sitting',
    'F01': 'fall',
    'F02': 'fall',
    'F06': 'fall',
    'F05': 'fall'
}

# 데이터 폴더 경로
data_folder = '../data/SisFall_dataset'

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        # user_ID가 'SA' 또는 'SE'로 시작하고 뒤에 오는 숫자가 7이나 8인 경우 처리
        if user_ID.startswith('SA') or user_ID.startswith('SE'):
            numeric_part = user_ID[2:]
            if numeric_part.isdigit() and int(numeric_part) in [7, 8]:
                for file_name in os.listdir(dir_path):
                    if file_name.endswith('.txt'):
                        # 파일 경로
                        file_path = os.path.join(dir_path, file_name)

                        # 파일 이름에 대한 정보
                        activity, _, trial_ID_with_extension = file_name.split('_')
                        trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                        if activity in ['D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'F01', 'F02', 'F05', 'F06']:
                            # task_id에 해당하는 label 정보
                            task_ID = activity
                            label = label_mapping[task_ID]

                            # Angular velocity 변환을 위한 공식에 사용되는 값들
                            Range = 2000  # 예시 값을 사용
                            Resolution = 16  # 예시 값을 사용

                            range_g = 16  # range for ADXL345 when set to ±16g
                            resolution_bits = 13  # resolution for ADXL345

                            # 파일에서 데이터를 읽어와서 처리
                            with open(file_path, 'r', encoding='utf-8') as file:
                                lines = file.readlines()

                            # 데이터 파싱 및 계산
                            gx_values = []
                            gy_values = []
                            gz_values = []

                            ax_values = []
                            ay_values = []
                            az_values = []

                            # 데이터 파싱 및 DataFrame에 추가
                            for i, line in enumerate(lines):
                                if i % 5 == 0:  # 10의 배수인 행만 처리
                                    # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                                    fields = line.strip().split(',')

                                    if len(fields) >= 6:  # 필드 수 확인
                                        # gx, gy, gz 값 얻기 및 변환 계산
                                        gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                                        gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                                        gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                                        ax_value = (2 * range_g / (2 ** resolution_bits)) * float(fields[0])
                                        ay_value = (2 * range_g / (2 ** resolution_bits)) * float(fields[1])
                                        az_value = (2 * range_g / (2 ** resolution_bits)) * float(fields[2])

                                        gx_values.append(gx_value)
                                        gy_values.append(gy_value)
                                        gz_values.append(gz_value)

                                        ax_values.append(ax_value)
                                        ay_values.append(ay_value)
                                        az_values.append(az_value)

                                        if len(gx_values) == 1:
                                            # 한 줄의 레코드 처리가 완료되었으므로 DataFrame에 추가
                                            df_val.loc[len(df_val)] = {
                                                'user_ID': user_ID,
                                                'trial_ID': trial_ID,
                                                'task_ID': task_ID,
                                                'gx': gx_values[0],
                                                'gy': gy_values[0],
                                                'gz': gz_values[0],
                                                'ax': ax_values[0],
                                                'ay': ay_values[0],
                                                'az': az_values[0],
                                                'label': label
                                            }
                                            record_count += 1

                                            gx_values = []
                                            gy_values = []
                                            gz_values = []

                                            ax_values = []
                                            ay_values = []
                                            az_values = []

# MinMaxScaler 인스턴스 생성
scaler = MinMaxScaler(feature_range=(-1, 1))

# 'gx', 'gy', 'gz', 'ax', 'ay', 'az' 열의 데이터 추출
data = df_val[['gx', 'gy', 'gz', 'ax', 'ay', 'az']].values

# 데이터 스케일링
scaled_data = scaler.fit_transform(data)

# 스케일링된 값을 새로운 열에 저장
df_val['scaled_gx'] = scaled_data[:, 0]
df_val['scaled_gy'] = scaled_data[:, 1]
df_val['scaled_gz'] = scaled_data[:, 2]
df_val['scaled_ax'] = scaled_data[:, 3]
df_val['scaled_ay'] = scaled_data[:, 4]
df_val['scaled_az'] = scaled_data[:, 5]

# CSV 파일로 저장
csv_path = '../data/SisFall_val_L_5.csv'
df_val.to_csv(csv_path, index=False)
print('CSV 파일이 저장되었습니다:', csv_path)

CSV 파일이 저장되었습니다: ../data/SisFall_val_L_20.csv


In [2]:
import os
import pandas as pd

# DataFrame 초기화
df_val = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz', 'label'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D02': 'walking',
    'D03': 'jogging',
    'D04': 'jogging',
    'D05': 'stairs_walking',
    'D06': 'stairs_walking',
    'D07': 'standing-sitting',
    'D08': 'standing-sitting',
    'F01': 'fall',
    'F02': 'fall',
    'F06': 'fall',
    'F05': 'fall'
}

# 데이터 폴더 경로
data_folder = '../data/SisFall_dataset'

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        # user_ID가 'SA' 또는 'SE'로 시작하고 뒤에 오는 숫자가 6이거나 7인 경우에만 처리
        if (user_ID.startswith('SA') or user_ID.startswith('SE')) and user_ID[2:].isdigit() and int(user_ID[2:]) in [6, 7]:
            for file_name in os.listdir(dir_path):
                if file_name.endswith('.txt'):
                    # 파일 경로
                    file_path = os.path.join(dir_path, file_name)

                    # 파일 이름에 대한 정보
                    activity, _, trial_ID_with_extension = file_name.split('_')
                    trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                    # task_id가 D01, D04, D05, D07, F05, F06인 경우에만 처리
                    if activity in ['D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'F01', 'F02', 'F05', 'F06']:
                        # task_id에 해당하는 label 정보
                        task_ID = activity
                        label = label_mapping[task_ID]

                        # Angular velocity 변환을 위한 공식에 사용되는 값들
                        Range = 2000  # 예시 값을 사용
                        Resolution = 16  # 예시 값을 사용

                        # 파일에서 데이터를 읽어와서 처리
                        with open(file_path, 'r', encoding='utf-8') as file:
                            lines = file.readlines()

                        # 데이터 파싱 및 계산
                        gx_values = []
                        gy_values = []
                        gz_values = []

                        # 데이터 파싱 및 DataFrame에 추가
                        for i, line in enumerate(lines):
                            if i % 10 == 0:  # 10의 배수인 행만 처리
                                # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                                fields = line.strip().split(',')

                                if len(fields) >= 6:  # 필드 수 확인
                                    # gx, gy, gz 값 얻기 및 변환 계산
                                    gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                                    gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                                    gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                                    gx_values.append(gx_value)
                                    gy_values.append(gy_value)
                                    gz_values.append(gz_value)

                                    if len(gx_values) == 1:
                                        # 한 줄의 레코드 처리가 완료되었으므로 DataFrame에 추가
                                        df_val.loc[len(df_val)] = {
                                            'user_ID': user_ID,
                                            'trial_ID': trial_ID,
                                            'task_ID': task_ID,
                                            'gx': gx_values[0],
                                            'gy': gy_values[0],
                                            'gz': gz_values[0],
                                            'label': label
                                        }
                                        record_count += 1
                                        gx_values = []
                                        gy_values = []
                                        gz_values = []

print(df_val)

      user_ID trial_ID task_ID        gx         gy         gz   
0        SA06      R01     D01 -0.061035  54.199219  19.470215  \
1        SA06      R01     D01 -5.981445  52.062988  27.465820   
2        SA06      R01     D01  0.366211  59.143066  20.141602   
3        SA06      R01     D01  7.080078  45.654297  26.916504   
4        SA06      R01     D01 -0.427246  40.832520  26.367188   
...       ...      ...     ...       ...        ...        ...   
77095    SE07      R05     D08 -0.549316   4.089355  -0.732422   
77096    SE07      R05     D08 -1.037598   4.211426  -0.427246   
77097    SE07      R05     D08 -0.427246   4.516602  -0.366211   
77098    SE07      R05     D08 -0.854492   5.004883  -0.244141   
77099    SE07      R05     D08 -1.098633   4.943848   0.183105   

                  label  
0               walking  
1               walking  
2               walking  
3               walking  
4               walking  
...                 ...  
77095  standing-sitting  

In [3]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 인스턴스 생성
scaler = MinMaxScaler(feature_range=(-1, 1))

# 'gx', 'gy', 'gz' 열의 데이터 추출
data = df_val[['gx', 'gy', 'gz']].values

# 데이터 스케일링
scaled_data = scaler.fit_transform(data)

# 스케일링된 값을 새로운 열에 저장
df_val['scaled_gx'] = scaled_data[:, 0]
df_val['scaled_gy'] = scaled_data[:, 1]
df_val['scaled_gz'] = scaled_data[:, 2]

print(df_val.head())

  user_ID trial_ID task_ID        gx         gy         gz    label   
0    SA06      R01     D01 -0.061035  54.199219  19.470215  walking  \
1    SA06      R01     D01 -5.981445  52.062988  27.465820  walking   
2    SA06      R01     D01  0.366211  59.143066  20.141602  walking   
3    SA06      R01     D01  7.080078  45.654297  26.916504  walking   
4    SA06      R01     D01 -0.427246  40.832520  26.367188  walking   

   scaled_gx  scaled_gy  scaled_gz  
0   0.153615   0.122234   0.097170  
1   0.145180   0.119433   0.107885  
2   0.154224   0.128716   0.098070  
3   0.163790   0.111031   0.107149  
4   0.153094   0.104709   0.106413  


In [ ]:
print(df_val['user_ID'].unique())

In [6]:
# CSV 파일로 저장
csv_path = '../data/SisFall_val_L.csv'
df_val.to_csv(csv_path, index=False)
print('CSV 파일이 저장되었습니다:', csv_path)

CSV 파일이 저장되었습니다: ../data/SisFall_test.csv


In [19]:
import os
import pandas as pd

# DataFrame 초기화
df_test = pd.DataFrame(columns=['user_ID', 'trial_ID', 'task_ID', 'gx', 'gy', 'gz', 'label'])

# task_id와 매핑되는 label 정보 생성
label_mapping = {
    'D01': 'walking',
    'D02': 'walking',
    'D03': 'jogging',
    'D04': 'jogging',
    'D05': 'stairs_walking',
    'D06': 'stairs_walking',
    'D07': 'standing-sitting',
    'D08': 'standing-sitting',
    'F01': 'fall',
    'F02': 'fall',
    'F06': 'fall',
    'F05': 'fall'
}

# 데이터 폴더 경로
data_folder = '../data/SisFall_dataset'

# 폴더 내의 파일들을 확인하고 처리
record_count = 0  # 전체 파일에서의 레코드 개수
for root, dirs, files in os.walk(data_folder):
    for dir_name in dirs:
        dir_path = os.path.join(root, dir_name)
        user_ID = dir_name

        # user_ID가 'SA' 또는 'SE'로 시작하고 뒤에 오는 숫자가 6이거나 7인 경우에만 처리
        if (user_ID.startswith('SA') or user_ID.startswith('SE')) and user_ID[2:].isdigit() and int(user_ID[2:]) in [8, 9]:
            for file_name in os.listdir(dir_path):
                if file_name.endswith('.txt'):
                    # 파일 경로
                    file_path = os.path.join(dir_path, file_name)

                    # 파일 이름에 대한 정보
                    activity, _, trial_ID_with_extension = file_name.split('_')
                    trial_ID = trial_ID_with_extension[:-4]  # .txt 제거

                    # task_id가 D01, D04, D05, D07, F05, F06인 경우에만 처리
                    if activity in ['D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'F01', 'F02', 'F05', 'F06']:
                        # task_id에 해당하는 label 정보
                        task_ID = activity
                        label = label_mapping[task_ID]

                        # Angular velocity 변환을 위한 공식에 사용되는 값들
                        Range = 2000  # 예시 값을 사용
                        Resolution = 16  # 예시 값을 사용

                        # 파일에서 데이터를 읽어와서 처리
                        with open(file_path, 'r', encoding='utf-8') as file:
                            lines = file.readlines()

                        # 데이터 파싱 및 계산
                        gx_values = []
                        gy_values = []
                        gz_values = []

                        # 데이터 파싱 및 DataFrame에 추가
                        for i, line in enumerate(lines):
                            if i % 10 == 0:  # 10의 배수인 행만 처리
                                # 쉼표를 기준으로 줄을 분리하여 필드의 값을 가져오기
                                fields = line.strip().split(',')

                                if len(fields) >= 6:  # 필드 수 확인
                                    # gx, gy, gz 값 얻기 및 변환 계산
                                    gx_value = (2 * Range / (2 ** Resolution)) * float(fields[3])
                                    gy_value = (2 * Range / (2 ** Resolution)) * float(fields[4])
                                    gz_value = (2 * Range / (2 ** Resolution)) * float(fields[5])

                                    gx_values.append(gx_value)
                                    gy_values.append(gy_value)
                                    gz_values.append(gz_value)

                                    if len(gx_values) == 1:
                                        # 한 줄의 레코드 처리가 완료되었으므로 DataFrame에 추가
                                        df_test.loc[len(df_test)] = {
                                            'user_ID': user_ID,
                                            'trial_ID': trial_ID,
                                            'task_ID': task_ID,
                                            'gx': gx_values[0],
                                            'gy': gy_values[0],
                                            'gz': gz_values[0],
                                            'label': label
                                        }
                                        record_count += 1
                                        gx_values = []
                                        gy_values = []
                                        gz_values = []

print(df_test)

       user_ID trial_ID task_ID         gx         gy         gz     label
0         SA01      R01     D01  -1.098633 -30.761719 -21.484375   walking
1         SA01      R01     D01 -10.864258 -46.752930  -3.173828   walking
2         SA01      R01     D01  31.860352 -22.216797   8.056641   walking
3         SA01      R01     D01   2.624512 -11.352539  29.052734   walking
4         SA01      R01     D01   7.263184  15.869141  26.184082   walking
...        ...      ...     ...        ...        ...        ...       ...
331797    SE15      R05     D07  -1.281738   3.601074  -0.366211  standing
331798    SE15      R05     D07  -1.586914   3.295898  -0.610352  standing
331799    SE15      R05     D07  -1.586914   3.173828  -0.915527  standing
331800    SE15      R05     D07  -1.525879   2.868652  -0.976562  standing
331801    SE15      R05     D07  -1.281738   3.051758  -0.915527  standing

[331802 rows x 7 columns]


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# MinMaxScaler 인스턴스 생성
scaler = MinMaxScaler(feature_range=(-1, 1))

# 'gx', 'gy', 'gz' 열의 데이터 추출
data = df_test[['gx', 'gy', 'gz']].values

# 데이터 스케일링
scaled_data = scaler.fit_transform(data)

# 스케일링된 값을 새로운 열에 저장
df_test['scaled_gx'] = scaled_data[:, 0]
df_test['scaled_gy'] = scaled_data[:, 1]
df_test['scaled_gz'] = scaled_data[:, 2]

print(df_test.head())

In [21]:
# CSV 파일로 저장
csv_path = '../data/SisFall_test_L.csv'
df.to_csv(csv_path, index=False)
print('CSV 파일이 저장되었습니다:', csv_path)

CSV 파일이 저장되었습니다: ../data/SisFall_train.csv
